In [1]:
import os
import dotenv
from pathlib import Path

from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import (
    WebBaseLoader, 
    PyPDFLoader, 
    Docx2txtLoader,
)
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

dotenv.load_dotenv()

USER_AGENT environment variable not set, consider setting it to identify your requests.


True

In [2]:
# Load docs

doc_paths = [
    "docs/가스계통_운영규정.pdf",
    "docs/여비규정.pdf",
    "docs/취업규칙.pdf",
]

docs = [] 
for doc_file in doc_paths:
    file_path = Path(doc_file)

    try:
        if doc_file.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        elif doc_file.endswith(".docx"):
            loader = Docx2txtLoader(file_path)
        elif doc_file.endswith(".txt") or doc_file.name.endswith(".md"):
            loader = TextLoader(file_path)
        else:
            print(f"Document type {doc_file.type} not supported.")
            continue

        docs.extend(loader.load())

    except Exception as e:
        print(f"Error loading document {doc_file.name}: {e}")



In [3]:
docs

[Document(metadata={'source': 'docs/가스계통_운영규정.pdf', 'page': 0}, page_content="제1조(목적)이 규정은 한국가스공사 (이하 “공사”라 한다)의 천연가스 (이하 “가스”라 한\n다)계통의 안전 ·안정적 운영에 관한 기준과 절차를 정함을 목적으로 한다.<개정 '16. 6. \n30.> \n제2조(적용범위 )공사가 운영하는 가스계통에 관하여 관계법령에 특별한 규정이 있는 \n경우를 제외하고는 이 규정이 정하는 바에 의한다.<개정 '16. 6. 30.> \n제3조(용어의 정의 )이 규정에서 사용하는 용어의 정의는 다음 각 호와 같다. <개정 \n'95.12. 1., ‘12. 5.21., ’16. 6. 30.> \n1. “가스계통 ”이라 함은 생산기지에서 생산한 가스를 수요자에게 공급하기 위하여 물리\n적으로 연결된 체계를 말하며, 생산 및 공급계통으로 구분한다.<신설 ‘16. 6. 30.> \n2. “수요자”라 함은 천연가스 공급규정의 수요자를 말한다.<신설 ‘16. 6. 30.> \n3. “생산계통 ”이라 함은 생산기지에서 가스를 송출하기 위하여 물리적으로 연결된 체계\n를 말한다.<신설 ‘16. 6. 30.> \n4. “공급계통 ”이라 함은 생산계통에서 송출된 가스를 수요자에게 공급하기 위하여 물리\n적으로 연결된 체계를 말한다.<신설 ‘16. 6. 30.> \n5. “중앙통제소 ”라 함은 본사내 전국통제설비를 이용하여 가스계통 운영을 총괄하는 \n24시간 상시 운영조직을 말한다. <개정 '98. 9. 4., '10. 3. 23., ‘12. 5.21., ’16. 6. 30.> \n6. “지역통제소 ”라 함은 해당 지역본부의 공급계통을 운영하는 24시간 상시 운영조직\n을 말한다. <개정 '98. 9. 4., '10. 3. 23., ‘12. 5.21., ’16. 6. 30.> \n7. “기지조정실 ”이라 함은 해당 기지본부의 생산계통을 운영하는 24시간 상시 운영조\n직을 말한다. <개정 ‘12. 5.21.

In [4]:
# Split docs

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap=1000,
)

document_chunks = text_splitter.split_documents(docs)

In [5]:
# Tokenize and load the documents to the vector store

vector_db = Chroma.from_documents(
    documents=document_chunks,
    embedding=OpenAIEmbeddings(),
)

In [6]:
# Retrieve

def _get_context_retriever_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
        ("user", "Given the above conversation, generate a search query to look up in order to get inforamtion relevant to the conversation, focusing on the most recent messages."),
    ])
    retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

    return retriever_chain

In [7]:
def get_conversational_rag_chain(llm):
    retriever_chain = _get_context_retriever_chain(vector_db, llm)

    prompt = ChatPromptTemplate.from_messages([
        ("system",
        """You are an assistant designed specifically for answering queries based on company regulations. Always respond strictly according to the company's internal regulations, ensuring your answers are aligned with these rules. 
        When providing an answer, first cite the most relevant regulation in detail, including chapter and section numbers if applicable. If multiple regulations apply, list all relevant ones before giving your response. 
        Your goal is to provide the user with clear guidance based on the regulations, so be as specific as possible with the details of the rules and regulations before proceeding with the final answer.
        If no regulation directly applies, inform the user and give your best guidance based on your knowledge of the company's practices.\n
        {context}"""),
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
    ])
    stuff_documents_chain = create_stuff_documents_chain(llm, prompt)

    return create_retrieval_chain(retriever_chain, stuff_documents_chain)

In [8]:
# Augmented Generation

llm_stream_openai = ChatOpenAI(
    model="gpt-4o",  # Here you could use "o1-preview" or "o1-mini" if you already have access to them
    temperature=0.1,
    streaming=True,
)

llm_stream_anthropic = ChatAnthropic(
    model="claude-3-5-sonnet-20240620",
    temperature=0.1,
    streaming=True,
)

llm_stream = llm_stream_openai  # Select between OpenAI and Anthropic models for the response

messages = [
    {"role": "user", "content": "안녕"},
    {"role": "assistant", "content": "안녕? 오늘 내가 뭘 도와줄까?"},
    {"role": "user", "content": "작업을 위하여 계통운영절차에 관한 기안문을 작성 중인데 주배관 밸브조작이 필요할 것 같아. 기안문에 주배관 조작 권한에 관한 규정을 관련근거로 넣고 싶은데, 이에 관한 규정이 어떤 것이 있어? 그리고 누구로부터 밸브조작 승인을 받아야 하는지 말해줄래?"},
]
messages = [HumanMessage(content=m["content"]) if m["role"] == "user" else AIMessage(content=m["content"]) for m in messages]

conversation_rag_chain = get_conversational_rag_chain(llm_stream)
response_message = "*(RAG Response)*\n"
for chunk in conversation_rag_chain.pick("answer").stream({"messages": messages[:-1], "input": messages[-1].content}):
    response_message += chunk
    print(chunk, end="", flush=True)

messages.append({"role": "assistant", "content": response_message})

주배관 밸브 조작에 관한 규정은 다음과 같습니다:

1. **제20조(배관 및 기기사고시 조작)**:
   - 제20조 제2항: "공급관리소 운전원 및 주배관 순찰원은 천재지변 또는 이에 준하는 경우로서 가스배관의 긴급사고 발생시 구간 차단조작을 수행할 수 있으며, 그 상황을 해당 지역통제소로 즉시 보고하여야 한다."
   - 제20조 제5항: "배관사고시 가스공급량 조절이 불가피한 경우, 생산기지 계통설비 조작은 가스계통 주관부서장의 지령에 따라야 한다."

이 규정에 따르면, 주배관의 밸브 조작은 긴급사고 발생 시 공급관리소 운전원 및 주배관 순찰원이 수행할 수 있으며, 상황을 즉시 지역통제소에 보고해야 합니다. 또한, 가스공급량 조절이 필요한 경우에는 가스계통 주관부서장의 지령에 따라야 합니다.

따라서, 주배관 밸브 조작을 위해서는 **가스계통 주관부서장의 승인을 받아야** 합니다. 기안문에 이 규정을 관련 근거로 포함시키면 됩니다.

## Multi-Query Retriever

In [2]:
# Build a sample vectorDB
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from pathlib import Path
import shutil

# Load blog post
# Load docs

doc_paths = [
    "docs/가스계통_운영규정.pdf",
    "docs/여비규정.pdf",
    "docs/취업규칙.pdf",
]

docs = [] 
for doc_file in doc_paths:
    file_path = Path(doc_file)

    print("doc_file", doc_file)
    try:
        if doc_file.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        elif doc_file.endswith(".docx"):
            loader = Docx2txtLoader(file_path)
        elif doc_file.endswith(".txt") or doc_file.name.endswith(".md"):
            loader = TextLoader(file_path)
        else:
            print(f"Document type {doc_file.type} not supported.")
            continue

        docs.extend(loader.load())

    except Exception as e:
        print(f"Error loading document {doc_file.name}: {e}")


# Split docs

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap=1000,
)

document_chunks = text_splitter.split_documents(docs)

# VectorDB
model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
ko_embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs,
)

# Delete existing Chroma collection (if it exists)
persist_directory = "chroma_db"
if Path(persist_directory).exists():
    shutil.rmtree(persist_directory)

# Create a new Chroma collection
vectordb = Chroma.from_documents(
    documents=document_chunks,
    embedding=ko_embedding,
    persist_directory=persist_directory
)

# Persist the database
vectordb.persist()
print("VectorDB created and persisted successfully.")

USER_AGENT environment variable not set, consider setting it to identify your requests.


doc_file docs/가스계통_운영규정.pdf
doc_file docs/여비규정.pdf
doc_file docs/취업규칙.pdf


/Users/jwson/anaconda3/envs/kongasAI/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


VectorDB created and persisted successfully.


In [4]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chat_models import ChatOpenAI

# question = "작업을 위하여 계통운영절차에 관한 기안문을 작성 중인데 주배관 밸브조작이 필요할 것 같아. 기안문에 주배관 조작 권한에 관한 규정을 관련근거로 넣고 싶은데, 이에 관한 규정이 어떤 것이 있어? 그리고 누구로부터 밸브조작 승인을 받아야 하는지 말해줄래?"
question = "전근"
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)

In [5]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)

/var/folders/8x/vf4phpq52j5fn3k5cs2sh9fw0000gn/T/ipykernel_35288/4025390597.py:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  unique_docs = retriever_from_llm.get_relevant_documents(query=question)
INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are some documents related to the topic of "전근"?', '2. Can you provide information on the concept of "전근"?', '3. Are there any resources that discuss the significance of "전근"?']


6

In [6]:
unique_docs

[Document(metadata={'page': 20, 'source': 'docs/여비규정.pdf'}, page_content='[별표 10]\n이 주 확 인 서\n(제23조 관련 )\n<신설 2017.1.25>\n1.사 번 :\n2.성 명 :\n3.직 급 :\n4.직 위 :\n5.소 속 :\n6.주 소\n-전 거주지 :\n-신 거주지 :\n위 직원은 인사발령 제 호(년 월 일)에 의거 전 거주지에서 신 거주지로 이전하여   \n현재 거주하고 있으며 ,본 확인서가 허위로 작성된 문서임이 판명될 경우 전근여비 \n전액을 환급하며 그에 대해 책임질 것을 확인합니다 .\n20 . . .\n이주자 (인)\n확인자 (소속부서장 ) (인)\n(주)이주자의 직위가 부서장 이상일 경우 ,직상위자가 확인한다 .'),
 Document(metadata={'page': 14, 'source': 'docs/취업규칙.pdf'}, page_content='[별표3]\n소\n속\n부\n서담당과장부장 주\n관\n부\n서담당과장부장\n지참(조퇴)계\n1.사유\n2.기간20년월일시분부터\n 시분까지\n위와같이불가피지참(조퇴)하였(겠)기에제출합니다.\n20년월일\n소속:\n직위:\n성명:(인)'),
 Document(metadata={'page': 15, 'source': 'docs/취업규칙.pdf'}, page_content='[별표4]\n시간외명령부\n═══════════\n직\n위성\n명\n월일명령자시간수명자시간외\n근무사항확인'),
 Document(metadata={'page': 13, 'source': 'docs/취업규칙.pdf'}, page_content='[별표2]\n소\n속\n부\n서담당과장부장 주\n관\n부\n서담당과장부장\n결근계유결\n 병결\n1.사유\n2.기간20년월일부터\n20년월일까지(일간)\n3.여행시의행선지\n위와같이출근하지못하겠으므로이에제출합니다.\n20년월일\n소속:\n직위:\n성명:(인)'),
 Document(metadata={'page': 19, 's

In [33]:
from langchain.chains import RetrievalQA

# Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever_from_llm,
    return_source_documents=True
)

# Get the answer
result = qa_chain({"query": question})

# Print the answer and sources
print("Answer:", result["result"])
print("\nSources:")
for doc in result["source_documents"]:
    print(f"- {doc.metadata.get('source', 'Unknown source')}")
    print(f"  Content: {doc.page_content[:200]}...")  # Print first 200 characters of each source

/var/folders/8x/vf4phpq52j5fn3k5cs2sh9fw0000gn/T/ipykernel_4514/3830042524.py:11: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})
INFO:langchain.retrievers.multi_query:Generated queries: ['어떤 규정이 주배관 조작 권한에 관한 기안문에 포함되어야 할까?', '밸브조작 승인을 받기 위해 어떤 단계를 거쳐야 할까?', '주배관 밸브조작 권한에 대한 규정은 누구로부터 승인을 받아야 하는가?']


Answer: 주배관 밸브조작과 관련된 규정은 "가스계통운영규정"에 있습니다. 이 규정에 따르면, 광역 주배관의 밸브조작은 가스계통 주관부서장의 지령 또는 승인에 따라 이루어져야 합니다. 따라서, 주배관 밸브조작 권한은 가스계통 주관부서장에게 있으며, 해당 부서장으로부터 승인을 받아야 합니다.

Sources:
- docs/여비규정.pdf
  Content: [별표 9]
제  회 공무국외여행 사전심사 서면결의서
 <신설 2011.11.29 >
◦부의안건  
-제  호 : (◯◯◯◯◯팀)
여비규정 제 26조의 5제 2항의 규정에 의거 위의 안건을 서면결의 방식으로 처리 
요청하오니 동의 여부를 해당란에 서명하여 주시기 바랍니다 .
위원성명 서면결의 서면심사 결과
서  명
직위 성명 동의부동의 원안의결 수정의결 보...
- docs/취업규칙.pdf
  Content: 취업규칙
[시행일자 : 2024-01-31]
제1장 총 칙 
제1조(목적) 이 규칙은 근로기준법에 따라 직원의 일상취업에 관한 사항을 정함을 목적 으로 한다. 
제2조(적용범위 ) 직원의 취업에 관여하는 이 규칙이 정하는 바에 의하며 이 규칙이 정 하지 아니한 사항은 근로기준
법, 관계법령 , 정관 및 단체협약에 따른다 . 적용과정에서 공사의 인권경영헌장과...
- docs/여비규정.pdf
  Content: 경일로 한다.
부 칙
이 규정은 2018년 6월 11일부터 시행한다 .
부 칙 <2019.10.25>
이 규정은 2019년 6월 30일로부터 시행한다 .
부 칙 <2023.10.20>
제1조(시행일 ) 이 규정은 2023년 10월 20일로부터 시행한다 .
제2조(경과조치 ) 이 규정 제12조, 별표1, 별표2, 별표3에 관한 사항은 그 시행일을 출장관
리시...
- docs/여비규정.pdf
  Content: [별표 7]
공무국외여행 사전심사 부의안건 서식
<신설 2010.6.28 >
    의안번호 제       호 의  
결  
사  
항의   결
년  월  일20 ...
(제     회)


In [26]:
prompt = ChatPromptTemplate.from_messages([
    ("system",
    """You are an assistant designed specifically for answering queries based on company regulations. Always respond strictly according to the company's internal regulations, ensuring your answers are aligned with these rules. 
    When providing an answer, first cite the most relevant regulation in detail, including chapter and section numbers if applicable. If multiple regulations apply, list all relevant ones before giving your response. 
    Your goal is to provide the user with clear guidance based on the regulations, so be as specific as possible with the details of the rules and regulations before proceeding with the final answer.
    If no regulation directly applies, inform the user and give your best guidance based on your knowledge of the company's practices.\n
    {context}"""),
    MessagesPlaceholder(variable_name="messages"),
    ("user", "{input}"),
])

llm_chain = prompt | llm

In [27]:
retriever = MultiQueryRetriever(
    retriever=vectordb.as_retriever(), llm_chain=llm_chain
)  # "lines" is the key (attribute name) of the parsed output

In [31]:
messages = [
    {"role": "user", "content": "안녕"},
    {"role": "assistant", "content": "안녕? 오늘 내가 뭘 도와줄까?"},
    {"role": "user", "content": "작업을 위하여 계통운영절차에 관한 기안문을 작성 중인데 주배관 밸브조작이 필요할 것 같아. 기안문에 주배관 조작 권한에 관한 규정을 관련근거로 넣고 싶은데, 이에 관한 규정이 어떤 것이 있어? 그리고 누구로부터 밸브조작 승인을 받아야 하는지 말해줄래?"},
]
messages = [HumanMessage(content=m["content"]) if m["role"] == "user" else AIMessage(content=m["content"]) for m in messages]

retriever.invoke({"messages": messages[:-1], "input": messages[-1].content})

KeyError: "Input to ChatPromptTemplate is missing variables {'context', 'input', 'messages'}.  Expected: ['context', 'input', 'messages'] Received: ['question']\nNote: if you intended {context} to be part of the string and not a variable, please escape it with double curly braces like: '{{context}}'."

In [8]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [12]:
unique_docs = retriever_from_llm.invoke(input=question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['어떤 규정이 주배관 조작 권한에 관한 기안문에 포함되어야 할까?', '밸브조작 승인을 받기 위해 어떤 단계를 거쳐야 할까?', '주배관 밸브조작에 대한 규정 및 승인 요청은 누구에게 해야 할까?']


10

In [13]:
unique_docs

[Document(metadata={'page': 19, 'source': 'docs/여비규정.pdf'}, page_content='[별표 9]\n제  회 공무국외여행 사전심사 서면결의서\n <신설 2011.11.29 >\n◦부의안건  \n-제  호 : (◯◯◯◯◯팀)\n여비규정 제 26조의 5제 2항의 규정에 의거 위의 안건을 서면결의 방식으로 처리 \n요청하오니 동의 여부를 해당란에 서명하여 주시기 바랍니다 .\n위원성명 서면결의 서면심사 결과\n서  명\n직위 성명 동의부동의 원안의결 수정의결 보류 부결\n위원◯◯◯\n위원◯◯◯\n위원◯◯◯\n위원◯◯◯\n위원◯◯◯\n위원◯◯◯\n부위원장◯◯◯\n위원장◯◯◯\n※위 건을 다음과 같이 확인함 .\n가.원안의결\n나.별지와 같이 수정의결\n다.보류\n라.부결\n 20 ...\n 공무국외여행 사전심사위원회 위원장  ◯◯◯(인)'),
 Document(metadata={'page': 0, 'source': 'docs/취업규칙.pdf'}, page_content='취업규칙\n[시행일자 : 2024-01-31]\n제1장 총 칙 \n제1조(목적) 이 규칙은 근로기준법에 따라 직원의 일상취업에 관한 사항을 정함을 목적 으로 한다. \n제2조(적용범위 ) 직원의 취업에 관여하는 이 규칙이 정하는 바에 의하며 이 규칙이 정 하지 아니한 사항은 근로기준\n법, 관계법령 , 정관 및 단체협약에 따른다 . 적용과정에서 공사의 인권경영헌장과 인권경영규정을 존중하고 준수해야 한다. \n<개정 2019.06.27>\n제3조(근무형태 및 정의)\n① 이 규칙에서 특수근무라함은 교대근무를 말한다 . \n② ＂교대근무 ＂라 함은 계속 가동 또는 근무를 요하는 장소에서 교대제에 따라 근무하는 것을 말한다 . <2024.01.31>\n③ ＂통상근무 ＂라 함은 제2항 이외의 근무를 말한다 . <2024.01.31>\n제4조(용어의 정의) 이 규칙에서의 용어의 정의는 다음과 같다. \n1. ＂상사＂라 함은 직제 그 밖의 직무조직상 직접 또는 간접

## 기본 Parent-document Retriever

In [14]:
from langchain.retrievers import ParentDocumentRetriever

In [20]:
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain.embeddings import HuggingFaceEmbeddings


In [16]:

loaders = [
    PyPDFLoader("docs/가스계통_운영규정.pdf"),
    PyPDFLoader("docs/여비규정.pdf"),
    PyPDFLoader("docs/취업규칙.pdf"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load_and_split())

In [17]:
model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
ko_embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

/Users/jwson/anaconda3/envs/kongasAI/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [21]:
# This text splitter is used to create the child documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=ko_embedding
)
# The storage layer for the parent documents
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

INFO:backoff:Backing off send_request(...) for 0.5s (requests.exceptions.ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))


In [23]:
retriever.add_documents(docs, ids=None)

In [24]:
sub_docs = vectorstore.similarity_search("가스계통 운영규정")

In [25]:
print("글 길이: {}\n\n".format(len(sub_docs[0].page_content)))
print(sub_docs[0].page_content)

글 길이: 383


제8조(생산설비 운영 )
① 생산설비 운영부서는 가스계통 주관부서에서 작성한 일일 공급계획 및 송출계획에 
준하여 천연가스 생산설비를 운영하고, 가스계통 주관부서장이 지령한 송출조건에 따
라 생산기지의 압력, 유량 등을 조정한다. <개정 '95.12. 1., '98. 9. 4., '10. 3. 23., ‘12. Page 3 of 8 가스계통운영규정
2024-02-14 http://kogaslaw.kogas.or.kr:9115/lims/front/page/fulltext.html?pAct=print본 정보는 가스공사의 자산이며, 허락없이 무단복제 하거나 사용할 수 없습니다.KOGAS / 홍보실 홍보부 / 오정인 / 2024-02-14 11:31:55 / 172.***.***.73


In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader, TextLoader
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from pathlib import Path
import shutil

# 문서 로드 (이전과 동일)
doc_paths = [
    "docs/가스계통_운영규정.pdf",
    "docs/여비규정.pdf",
    "docs/취업규칙.pdf",
]

docs = []
for doc_file in doc_paths:
    file_path = Path(doc_file)
    try:
        if file_path.suffix == ".pdf":
            loader = PyPDFLoader(str(file_path))
        elif file_path.suffix == ".docx":
            loader = Docx2txtLoader(str(file_path))
        elif file_path.suffix in [".txt", ".md"]:
            loader = TextLoader(str(file_path))
        else:
            print(f"Document type {file_path.suffix} not supported.")
            continue
        docs.extend(loader.load())
    except Exception as e:
        print(f"Error loading document {file_path.name}: {e}")

# 개선된 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

document_chunks = text_splitter.split_documents(docs)

# 한국어에 더 적합한 임베딩 모델 사용
ko_embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v2")

# Chroma 벡터 저장소 생성
persist_directory = "improved_chroma_db"
if Path(persist_directory).exists():
    shutil.rmtree(persist_directory)

vectordb = Chroma.from_documents(
    documents=document_chunks,
    embedding=ko_embedding,
    persist_directory=persist_directory
)
vectordb.persist()

# LLM 설정
llm = ChatOpenAI(temperature=0)

# 컨텍스트 압축 검색기 설정
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_kwargs={"k": 5})
)

# 프롬프트 템플릿 (이전과 동일)
prompt_template = """
당신은 가스 회사의 규정 전문가입니다. 주어진 질문에 대해 관련 규정을 정확히 인용하여 답변해야 합니다.

질문: {question}

관련 문서의 내용:
{context}

위의 정보를 바탕으로 다음 형식에 맞춰 답변해주세요:
1. 먼저 "네," 또는 "죄송합니다,"로 시작하세요.
2. 관련 규정을 정확히 인용하세요 (예: 「가스계통운영규정」 제2장 가스계통운영 제6조(가스계통 운영) 2항).
3. 인용한 규정의 내용을 설명하세요.
4. 필요한 경우 추가 정보나 설명을 제공하세요.

답변:
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# RetrievalQA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type="stuff",
    chain_type_kwargs={"prompt": PROMPT}
)

# 질문 및 답변 얻기
question = "휴가를 다 사용하지 못할 것 같은데 1년에 저축 가능한 휴가가 며칠이야? 관련 규정도 같이 말해줘."
result = qa_chain({"query": question})

# 답변 및 출처 출력
print("Answer:", result["result"])
print("\nSources:")
for doc in result["source_documents"]:
    print(f"- {doc.metadata.get('source', 'Unknown source')}")
    print(f"  Content: {doc.page_content[:200]}...")

/var/folders/8x/vf4phpq52j5fn3k5cs2sh9fw0000gn/T/ipykernel_23756/3251841349.py:48: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  ko_embedding = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-nli")
/Users/jwson/anaconda3/envs/kongasAI/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/jwson/anaconda3/envs/kongasAI/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces`

Answer: 네, 연차휴가는 5년간 저축하여 사용할 수 있으며, 연간 10일 한도로 사용 가능합니다. 그러나 5년 이내 또는 퇴직 시까지 사용하지 않은 경우 저축 연차휴가는 자동으로 소멸됩니다. (근로기준법 제61조)

Sources:
- docs/취업규칙.pdf
  Content: ⑤ 연차휴가는 직원의 자유의사에 따라 적치하여 계산기간 만료 익일부터 1년 이내에 사용한다 . 단, 공사가 제29조 제1항, 
제3항 및 제4항에 따른 연차유급휴가 중 12일의 범위 내에 근로기준법 제61조에 따라 사용촉진조치를 시행하며 그 미사
용 휴가는 연간 10일 한도로 5년간 저축 사용 할 수 있으나 5년 이내 또는 퇴직 시까지 사용하지 아니한 저축...


In [4]:
# 질문 및 답변 얻기
question = "올해 초 정기인사 때 부산경남지역본부에서 본사로 발령이 났었는데 지금도 전근여비 신청이 가능할까?"
result = qa_chain({"query": question})

In [5]:
# 답변 및 출처 출력
print("Answer:", result["result"])
print("\nSources:")
for doc in result["source_documents"]:
    print(f"- {doc.metadata.get('source', 'Unknown source')}")
    print(f"  Content: {doc.page_content[:200]}...")

Answer: 죄송합니다, 본사로 발령된 후에 전근여비 신청이 가능한지에 대한 규정은 「근로규정」 제3장 근로계약 제12조(전근여비)에 명시되어 있습니다. 해당 규정에 따르면, 근로자가 근무지를 변경하게 되어 발생하는 전근여비는 사업주가 부담하여야 하며, 이에 대한 신청은 발령 후 1개월 이내에 해야 합니다. 따라서, 발령 후에도 전근여비 신청이 가능하며, 신청 기한을 엄수하는 것이 중요합니다. 추가적인 문의나 도움이 필요하시다면 해당 규정을 참고하시거나 인사팀에 문의해주시기 바랍니다.

Sources:


In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader, TextLoader
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from pathlib import Path
import shutil
from chromadb.config import Settings

# 문서 로드 및 처리 (이전과 동일)
doc_paths = [
    "docs/가스계통_운영규정.pdf",
    "docs/여비규정.pdf",
    "docs/취업규칙.pdf",
]

docs = []
for doc_file in doc_paths:
    file_path = Path(doc_file)
    try:
        if file_path.suffix == ".pdf":
            loader = PyPDFLoader(str(file_path))
        elif file_path.suffix == ".docx":
            loader = Docx2txtLoader(str(file_path))
        elif file_path.suffix in [".txt", ".md"]:
            loader = TextLoader(str(file_path))
        else:
            print(f"Document type {file_path.suffix} not supported.")
            continue
        docs.extend(loader.load())
    except Exception as e:
        print(f"Error loading document {file_path.name}: {e}")

# 텍스트 분할 (청크 크기 조정)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # 청크 크기를 줄임
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

document_chunks = text_splitter.split_documents(docs)

# 임베딩 모델 변경
ko_embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v2")

# Chroma 벡터 저장소 생성
persist_directory = "improved_chroma_db"
if Path(persist_directory).exists():
    shutil.rmtree(persist_directory)

vectordb = Chroma.from_documents(
    documents=document_chunks,
    embedding=ko_embedding,
    persist_directory=persist_directory
)
vectordb.persist()

# LLM 설정
llm = ChatOpenAI(temperature=0)

# 컨텍스트 압축 검색기 설정
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_kwargs={"k": 5})
)

# 프롬프트 템플릿 개선
prompt_template = """
당신은 가스 회사의 규정 전문가입니다. 주어진 질문에 대해 관련 규정을 정확히 인용하여 답변해야 합니다.
제공된 문서 내용에서만 정보를 찾아 답변하세요. 관련 정보가 없다면 "해당 질문에 대한 정보를 찾을 수 없습니다."라고 답변하세요.

질문: {question}

관련 문서의 내용:
{context}

위의 정보를 바탕으로 다음 형식에 맞춰 답변해주세요:
1. 먼저 "네," 또는 "죄송합니다,"로 시작하세요.
2. 관련 규정을 정확히 인용하세요 (예: 「가스계통운영규정」 제2장 가스계통운영 제6조(가스계통 운영) 2항).
3. 인용한 규정의 내용을 설명하세요.
4. 필요한 경우 추가 정보나 설명을 제공하세요.
5. 관련 정보가 없다면 "해당 질문에 대한 정보를 찾을 수 없습니다."라고 답변하세요.

답변:
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# RetrievalQA 체인 생성 (유사도 임계값 설정)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type="stuff",
    chain_type_kwargs={"prompt": PROMPT}
)

# 디버깅 정보를 포함한 질문 함수
def ask_question_with_debug(question):
    result = qa_chain({"query": question})
    print("질문:", question)
    print("\n답변:", result["result"])
    print("\n디버깅 정보:")
    for i, doc in enumerate(result["source_documents"]):
        print(f"문서 {i+1}:")
        print(f"- 출처: {doc.metadata.get('source', 'Unknown source')}")
        print(f"- 페이지: {doc.metadata.get('page', 'Unknown page')}")
        print(f"- 내용: {doc.page_content[:200]}...")
        print(f"- 유사도 점수: {doc.metadata.get('score', 'Unknown score')}")
        print()
    print("=" * 50 + "\n")

# 질문 처리
question = "올해 초 정기인사 때 부산경남지역본부에서 본사로 발령이 났었는데 지금도 전근여비 신청이 가능할까?"
ask_question_with_debug(question)

/var/folders/8x/vf4phpq52j5fn3k5cs2sh9fw0000gn/T/ipykernel_35288/1891840174.py:49: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  ko_embedding = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-nli")
/Users/jwson/anaconda3/envs/kongasAI/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/jwson/anaconda3/envs/kongasAI/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces`

질문: 올해 초 정기인사 때 부산경남지역본부에서 본사로 발령이 났었는데 지금도 전근여비 신청이 가능할까?

답변: 해당 질문에 대한 정보를 찾을 수 없습니다.

디버깅 정보:



In [2]:
ask_question_with_debug("전근 여비 신청에 대해 알려줘")

질문: 전근 여비 신청에 대해 알려줘

답변: 해당 질문에 대한 정보를 찾을 수 없습니다.

디버깅 정보:



In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader, TextLoader
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from pathlib import Path
import shutil
from chromadb.config import Settings
import re

# 문서 로드 및 처리
doc_paths = [
    "docs/가스계통_운영규정.pdf",
    "docs/여비규정.pdf",
    "docs/취업규칙.pdf",
]

docs = []
for doc_file in doc_paths:
    file_path = Path(doc_file)
    try:
        if file_path.suffix == ".pdf":
            loader = PyPDFLoader(str(file_path))
        elif file_path.suffix == ".docx":
            loader = Docx2txtLoader(str(file_path))
        elif file_path.suffix in [".txt", ".md"]:
            loader = TextLoader(str(file_path))
        else:
            print(f"Document type {file_path.suffix} not supported.")
            continue
        loaded_docs = loader.load()
        docs.extend(loaded_docs)
        print(f"Loaded {len(loaded_docs)} pages from {file_path.name}")
    except Exception as e:
        print(f"Error loading document {file_path.name}: {e}")

# 데이터 확인
print("\nData Verification:")
for doc in docs:
    print(f"Source: {doc.metadata.get('source')}")
    print(f"Page: {doc.metadata.get('page')}")
    print(f"Content preview: {doc.page_content[:200]}...\n")

# 텍스트 분할 (청크 크기 조정)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
    length_function=len,
    separators=["\n\n", "\n", ".", " ", ""]
)

document_chunks = text_splitter.split_documents(docs)

# 임베딩 모델 설정
ko_embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

# Chroma 벡터 저장소 생성 또는 로드
persist_directory = "./chroma_db"
client_settings = Settings(
    chroma_db_impl='duckdb+parquet',
    persist_directory=persist_directory,
    allow_reset=True
)

# 벡터 데이터베이스 생성 또는 로드
if not Path(persist_directory).exists():
    vectordb = Chroma.from_documents(
        documents=document_chunks,
        embedding=ko_embedding,
        persist_directory=persist_directory,
        client_settings=client_settings
    )
    vectordb.persist()
    print("New vector database created and persisted.")
else:
    vectordb = Chroma(
        persist_directory=persist_directory,
        embedding_function=ko_embedding,
        client_settings=client_settings
    )
    print("Existing vector database loaded.")

# LLM 설정
llm = ChatOpenAI(temperature=0)

# 컨텍스트 압축 검색기 설정
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_kwargs={"k": 5})
)

# 키워드 기반 검색 함수
def keyword_search(query, documents, top_k=5):
    keywords = re.findall(r'\w+', query.lower())
    scored_docs = []
    for doc in documents:
        score = sum(1 for keyword in keywords if keyword in doc.page_content.lower())
        scored_docs.append((doc, score))
    return sorted(scored_docs, key=lambda x: x[1], reverse=True)[:top_k]

# 프롬프트 템플릿
prompt_template = """
당신은 가스 회사의 규정 전문가입니다. 주어진 질문에 대해 관련 규정을 정확히 인용하여 답변해야 합니다.
제공된 문서 내용에서만 정보를 찾아 답변하세요. 관련 정보가 없다면 "해당 질문에 대한 정보를 찾을 수 없습니다."라고 답변하세요.

질문: {question}

관련 문서의 내용:
{context}

위의 정보를 바탕으로 다음 형식에 맞춰 답변해주세요:
1. 먼저 "네," 또는 "죄송합니다,"로 시작하세요.
2. 관련 규정을 정확히 인용하세요 (예: 「가스계통운영규정」 제2장 가스계통운영 제6조(가스계통 운영) 2항).
3. 인용한 규정의 내용을 설명하세요.
4. 필요한 경우 추가 정보나 설명을 제공하세요.
5. 관련 정보가 없다면 "해당 질문에 대한 정보를 찾을 수 없습니다."라고 답변하세요.

답변:
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# RetrievalQA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type="stuff",
    chain_type_kwargs={"prompt": PROMPT}
)

# 디버깅 정보를 포함한 질문 함수
def ask_question_with_debug(question):
    # 벡터 검색 결과
    result = qa_chain({"query": question})
    
    # 키워드 검색 결과
    keyword_results = keyword_search(question, document_chunks)
    
    print("질문:", question)
    print("\n벡터 검색 답변:", result["result"])
    
    print("\n벡터 검색 디버깅 정보:")
    for i, doc in enumerate(result["source_documents"]):
        print(f"문서 {i+1}:")
        print(f"- 출처: {doc.metadata.get('source', 'Unknown source')}")
        print(f"- 페이지: {doc.metadata.get('page', 'Unknown page')}")
        print(f"- 내용: {doc.page_content[:200]}...")
        print(f"- 유사도 점수: {doc.metadata.get('score', 'Unknown score')}")
        print()
    
    print("\n키워드 검색 결과:")
    for i, (doc, score) in enumerate(keyword_results):
        print(f"문서 {i+1}:")
        print(f"- 출처: {doc.metadata.get('source', 'Unknown source')}")
        print(f"- 페이지: {doc.metadata.get('page', 'Unknown page')}")
        print(f"- 내용: {doc.page_content[:200]}...")
        print(f"- 키워드 점수: {score}")
        print()
    
    print("=" * 50 + "\n")

# 질문 처리
question = "올해 초 정기인사 때 부산경남지역본부에서 본사로 발령이 났었는데 지금도 전근여비 신청이 가능할까?"
ask_question_with_debug(question)

Loaded 8 pages from 가스계통_운영규정.pdf
Loaded 23 pages from 여비규정.pdf
Loaded 16 pages from 취업규칙.pdf

Data Verification:
Source: docs/가스계통_운영규정.pdf
Page: 0
Content preview: 제1조(목적)이 규정은 한국가스공사 (이하 “공사”라 한다)의 천연가스 (이하 “가스”라 한
다)계통의 안전 ·안정적 운영에 관한 기준과 절차를 정함을 목적으로 한다.<개정 '16. 6. 
30.> 
제2조(적용범위 )공사가 운영하는 가스계통에 관하여 관계법령에 특별한 규정이 있는 
경우를 제외하고는 이 규정이 정하는 바에 의한다.<개정 '16. 6. 30...

Source: docs/가스계통_운영규정.pdf
Page: 1
Content preview: 16. 6. 30.> 
14. “계통설비 ”라 함은 가스계통에 영향을 주는 설비를 말한다. <개정 '98. 9. 4., ‘12. 
5.21., ’16. 6. 30.> 
15. “계통사고 ”라 함은 계통설비의 고장 또는 오조작으로 가스계통에 영향을 주거나 줄 
우려가 있는 사고를 말한다.<개정 ‘12. 5.21., ’16. 6. 30.> 
16. “계약공급압...

Source: docs/가스계통_운영규정.pdf
Page: 2
Content preview: 제6조의2(가스계통 주관부서 <개정 '10. 3. 23., ‘12. 5.21., ‘16. 6. 30.>)
① 가스계통 주관부서장은 가스계통을 감시 및 총괄하며 필요시 운영을 지시한다. < 신
설 '95.12. 1., 개정 '98. 9. 4., '10. 3. 23., ‘12. 5.21., ’16. 6. 30., 2018.12.13>
② 가스계통 주관부서장은 ...

Source: docs/가스계통_운영규정.pdf
Page: 3
Content preview: 5.21., ’16. 6. 30., 2023.06.27>
② 가스계통 주관부서장은 수요변동으로 인하여 생산설비 가

ValueError: [91mYou are using a deprecated configuration of Chroma.

[94mIf you do not have data you wish to migrate, you only need to change how you construct
your Chroma client. Please see the "New Clients" section of https://docs.trychroma.com/deployment/migration.
________________________________________________________________________________________________

If you do have data you wish to migrate, we have a migration tool you can use in order to
migrate your data to the new Chroma architecture.
Please `pip install chroma-migrate` and run `chroma-migrate` to migrate your data and then
change how you construct your Chroma client.

See https://docs.trychroma.com/deployment/migration for more information or join our discord at https://discord.gg/8g5FESbj for help![0m

In [6]:
from chromadb import PersistentClient, Settings

# Chroma 벡터 저장소 생성 또는 로드
persist_directory = "./chroma_db"
client_settings = Settings(
    is_persistent=True,
    persist_directory=persist_directory
)

# 클라이언트 초기화
chroma_client = PersistentClient(path=persist_directory, settings=client_settings)

# 컬렉션 생성 또는 가져오기
collection_name = "my_collection"
try:
    collection = chroma_client.get_collection(name=collection_name)
    print(f"Existing collection '{collection_name}' loaded.")
except ValueError:
    collection = chroma_client.create_collection(name=collection_name)
    print(f"New collection '{collection_name}' created.")

# Langchain Chroma 래퍼 초기화
vectordb = Chroma(
    client=chroma_client,
    collection_name=collection_name,
    embedding_function=ko_embedding
)

# 데이터가 없는 경우에만 문서 추가
if collection.count() == 0:
    vectordb.add_documents(document_chunks)
    print("Documents added to the collection.")
else:
    print("Collection already contains documents. Skipping document addition.")

New collection 'my_collection' created.
Documents added to the collection.
